In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

In [2]:
inputPath = "D:/college-lessons/MA/FIU/AI-workshop/dataset/Houses-dataset-master/Houses Dataset/HousesInfo.txt"
datasetPath = "D:/college-lessons/MA/FIU/AI-workshop/dataset/Houses-dataset-master/Houses Dataset"

In [3]:


cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)


zipcodes, counts = np.unique(df["zipcode"], return_counts=True)

# loop over each of the unique zip codes and their corresponding
# count
for (zipcode, count) in zip(zipcodes, counts):
    # the zip code counts for our housing dataset is *extremely*
    # unbalanced (some only having 1 or 2 houses per zip code)
    # so let's sanitize our data by removing any houses with less
    # than 25 houses per zip code
    if count < 25:
        idxs = df[df["zipcode"] == zipcode].index
        df.drop(idxs, inplace=True)

In [4]:
# initialize our images array (i.e., the house images themselves)
images = []

# loop over the indexes of the houses
for i in df.index.values:
    # find the four images for the house and sort the file paths,
    # ensuring the four are always in the *same order*
    basePath = os.path.sep.join([datasetPath, "{}_*".format(i + 1)])
    housePaths = sorted(list(glob.glob(basePath)))
    # initialize our list of input images along with the output image
    # after *combining* the four input images
    inputImages = []
    outputImage = np.zeros((64, 64, 3), dtype="uint8")

    # loop over the input house paths
    for housePath in housePaths:
        # load the input image, resize it to be 32 32, and then
        # update the list of input images
        image = cv2.imread(housePath)
        image = cv2.resize(image, (32, 32))
        inputImages.append(image)

    # tile the four input images in the output image such the first
    # image goes in the top-right corner, the second image in the
    # top-left corner, the third image in the bottom-right corner,
    # and the final image in the bottom-left corner
    outputImage[0:32, 0:32] = inputImages[0]
    outputImage[0:32, 32:64] = inputImages[1]
    outputImage[32:64, 32:64] = inputImages[2]
    outputImage[32:64, 0:32] = inputImages[3]

    # add the tiled image to our set of images the network will be
    # trained on
    images.append(outputImage)
images = np.array(images)

In [5]:
images = images / 255.0

In [6]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [7]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

In [8]:
# initialize the column names of the continuous data
continuous = ["bedrooms", "bathrooms", "area"]

# performin min-max scaling each continuous feature column to
# the range [0, 1]
cs = MinMaxScaler()
trainContinuous = cs.fit_transform(trainAttrX[continuous])
testContinuous = cs.transform(testAttrX[continuous])

# one-hot encode the zip code categorical data (by definition of
# one-hot encoing, all output features are now in the range [0, 1])
zipBinarizer = LabelBinarizer().fit(df["zipcode"])
trainCategorical = zipBinarizer.transform(trainAttrX["zipcode"])
testCategorical = zipBinarizer.transform(testAttrX["zipcode"])

# construct our training and testing data points by concatenating
# the categorical features with the continuous features
trainAttrX = np.hstack([trainCategorical, trainContinuous])
testAttrX = np.hstack([testCategorical, testContinuous])

In [9]:
dim = trainAttrX.shape[1] #10

# define our MLP network
mlp = Sequential()
mlp.add(Dense(8, input_dim=dim, activation="relu"))
mlp.add(Dense(4, activation="relu"))


In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization

width, height, depth = 64, 64, 3
filters=(16, 32, 64)
# initialize the input shape and channel dimension, assuming
# TensorFlow/channels-last ordering
inputShape = (height, width, depth)
chanDim = -1

# define the model input
inputs = Input(shape=inputShape)

# loop over the number of filters
for (i, f) in enumerate(filters):
    # if this is the first CONV layer then set the input
    # appropriately
    if i == 0:
        x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
# flatten the volume, then FC => RELU => BN => DROPOUT
x = Flatten()(x)
x = Dense(16)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=chanDim)(x)
x = Dropout(0.5)(x)

# apply another FC layer, this one to match the number of nodes
# coming out of the MLP
x = Dense(4)(x)
x = Activation("relu")(x)

# construct the CNN
cnn = Model(inputs, x)

In [15]:
from tensorflow.keras.layers import concatenate
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])

# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [16]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
 

In [17]:
# train the model
model.fit(
    [trainAttrX, trainImagesX], trainY,
    validation_data=([testAttrX, testImagesX], testY),
    epochs=200, batch_size=8)

Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 2s 8ms/sample - loss: 598.5432 - val_loss: 376.5008
Epoch 2/200
271/271 [==============================] - 0s 918us/sample - loss: 337.4662 - val_loss: 490.5604
Epoch 3/200
271/271 [==============================] - 0s 929us/sample - loss: 210.8032 - val_loss: 690.6719
Epoch 4/200
271/271 [==============================] - 0s 911us/sample - loss: 153.1075 - val_loss: 832.8999
Epoch 5/200
271/271 [==============================] - 0s 904us/sample - loss: 112.7116 - val_loss: 806.6903
Epoch 6/200
271/271 [==============================] - 0s 892us/sample - loss: 122.2779 - val_loss: 540.4234
Epoch 7/200
271/271 [==============================] - 0s 940us/sample - loss: 114.8352 - val_loss: 474.4300
Epoch 8/200
271/271 [==============================] - 0s 900us/sample - loss: 91.7487 - val_loss: 732.7233
Epoch 9/200
271/271 [==============================] - 0s 889us/sample - loss: 88.1069

271/271 [==============================] - 0s 911us/sample - loss: 22.0155 - val_loss: 30.2052
Epoch 152/200
271/271 [==============================] - 0s 907us/sample - loss: 21.4330 - val_loss: 27.8918
Epoch 153/200
271/271 [==============================] - 0s 922us/sample - loss: 21.4455 - val_loss: 24.4426
Epoch 154/200
271/271 [==============================] - 0s 926us/sample - loss: 21.3862 - val_loss: 24.1338
Epoch 155/200
271/271 [==============================] - 0s 922us/sample - loss: 24.1858 - val_loss: 23.8998
Epoch 156/200
271/271 [==============================] - 0s 896us/sample - loss: 21.6343 - val_loss: 24.0875
Epoch 157/200
271/271 [==============================] - 0s 900us/sample - loss: 21.5036 - val_loss: 25.7381
Epoch 158/200
271/271 [==============================] - 0s 911us/sample - loss: 22.0149 - val_loss: 24.3273
Epoch 159/200
271/271 [==============================] - 0s 900us/sample - loss: 19.9368 - val_loss: 24.4976
Epoch 160/200
271/271 [==========

In [18]:
# make predictions on the testing data
preds = model.predict([testAttrX, testImagesX])

In [19]:
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
 
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
 
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 27.04%, std: 28.41%


##  Contact me:


LinkedIn Account

https://www.linkedin.com/in/mohammed-behjoo-446098b7/

YouTube Channel

https://www.youtube.com/channel/UCudI0gvPh-YbiY2zLM7im-w

GitHub¶

https://github.com/mohammedbehjoo
